In [106]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from tornado.escape import json_decode
from tornado.gen import coroutine
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64

In [107]:
def encrypt_object(object, password):
    nonce_bytes = os.urandom(16)
    print(nonce_bytes)
    salt = os.urandom(16)
    print(salt)
    kdf = PBKDF2HMAC (
        algorithm=hashes.SHA256(),
        length=24,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    password_bytes = bytes(password, 'utf-8')
    key = base64.urlsafe_b64encode(kdf.derive(password_bytes))
    aes_ctr_cipher = Cipher(algorithms.AES(key), mode=modes.CTR(nonce_bytes))
    aes_ctr_encryptor = aes_ctr_cipher.encryptor()
    object_bytes = bytes(object, "utf-8")
    encrypted_object = aes_ctr_encryptor.update(object_bytes)
    return encrypted_object, nonce_bytes, salt

In [108]:
def hash_password(password):
    password_salt = os.urandom(16)
    kdf = Scrypt(salt=password_salt, length=32, n=2 ** 14, r=8, p=1)
    password_bytes = bytes(password, "utf-8")
    password_object = kdf.derive(password_bytes)
    key = str(password_object)
    return key, password_salt

In [109]:
email = 'test@barry.com'
password = 'password'

In [110]:
hashed_password, pw_salt = hash_password(password)
print(hashed_password)
print(pw_salt)

b'\xde^\x82\xb5CLq\r\x81\x003/\xab\xd8?L5\xb6\x91Tm\x1b-\x99q\xc4\xcby\x88\xfe\xfa\x9c'
b'\x8a\xef\n\xa0\x1d L\x17\xa8j\xf4c\x08\x81\xc4\xdf'


In [89]:
# hashed_password_bytes = bytes(hashed_password, "utf-8")

In [111]:
encrypted_email, enonce, esalt = encrypt_object(email, hashed_password)

b';* \xe6\xb5h\xfe$X9\xa27"\xe8\x98\xb9'
b"\xe9jb\xecj\xefz'\xc2\xf9\xe9\xf6yM@/"


In [112]:
print(encrypted_email)
print(enonce)
print(esalt)

b'\xaf\xcd5Z&\n.\x10\xfe\x86x)\xac^'
b';* \xe6\xb5h\xfe$X9\xa27"\xe8\x98\xb9'
b"\xe9jb\xecj\xefz'\xc2\xf9\xe9\xf6yM@/"


In [113]:
type(encrypted_email)

bytes

In [105]:
emailBytes = bytes(encrypted_email, 'utf-8')

TypeError: encoding without a string argument

In [114]:
def decrypt_object(object, password, nonce, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=24,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    password_bytes = bytes(password, 'utf-8')
    key = base64.urlsafe_b64encode(kdf.derive(password_bytes))
    aes_ctr_cipher = Cipher(algorithms.AES(key), mode=modes.CTR(nonce))
    aes_ctr_decryptor = aes_ctr_cipher.decryptor()
    decrypted_object = aes_ctr_decryptor.update(object)
    return decrypted_object

In [115]:
descrypted_email = decrypt_object(b'\xaf\xcd5Z&\n.\x10\xfe\x86x)\xac^', hashed_password, b';* \xe6\xb5h\xfe$X9\xa27"\xe8\x98\xb9', b"\xe9jb\xecj\xefz'\xc2\xf9\xe9\xf6yM@/")

In [116]:
descrypted_email

b'test@barry.com'

In [65]:
descrypted_email = descrypted_email.decode("utf-8")

In [66]:
descrypted_email

'test@barry.com'

In [67]:
2 ** 14

16384